Helper functions

In [10]:
import pandas as pd

def txt_to_pd(path):
    """takes path to data.txt returns pandas dataframe"""
    raw = open(path).readlines() 
    raw = [map(float, each.strip().split()) for each in raw] 
    df_data = list()
    for i in range(len(raw)):
        df_data.append(list(raw[i]))
    df1 = pd.DataFrame(df_data)
    return df1

Net Class

In [11]:
from torch import nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers:list, drop_p=0.2) -> None:
        ''' Builds a feedforward network with arbitrary hidden layers.
        
            Arguments
            ---------
            input_size: integer, size of the input layer
            output_size: integer, size of the output layer
            hidden_layers: list of integers, the sizes of the hidden layers
        '''
        super().__init__()
        # add input to hidden 
        self.layers = nn.ModuleList([nn.Linear(input_size, hidden_layers[0]).float()])
        #add hidden layers 
        layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
        self.layers.extend([nn.Linear(n1,n2) for n1,n2 in layer_sizes])
        #add output layer
        self.output = nn.Linear(hidden_layers[-1], output_size)
        #dropout
        self.dropout = nn.Dropout(p=drop_p)
    
    def forward(self, x):
        #flatten images
        x = x.view(x.shape[0], -1)
        
        #NN
        for layer in self.layers:
            x =  self.dropout(F.relu(layer(x)))
        return F.log_softmax(self.output(x), dim=1)

In [12]:
import torch
import numpy as np

def train(model, optimizer, criterion, train_loader, valid_loader, min_val_loss = np.Inf,  epochs=5):
    """
        Train model using train_loader data and validate on valid_loaders
        returns list of running and validation lost for each epoch 
        saves best configuration into model.pt

        after training call :
        model.load_state_dict(torch.load('model.pt')) to avoid overfitting  

        if used for transfer learning stuck data across dim =1 (RGB images)
    """
    train_plot, val_plot = [], []
    for epoch in range(epochs):
        train_loss, val_loss = 0,0

        #training loop
        model.train() 
        for data, target in train_loader:
            # resnet arhitecture expect data tensor acros 3 channels [3,150,150]
            # uncoment for approach 2
            #data = torch.stack([data,data,data], dim=1)

            optimizer.zero_grad()
            output = model(data)# forward pass
            target.squeeze_()
            loss = criterion(output, target)#calculate loss
            loss.backward() #calculate gradients 
            optimizer.step() # upddate weights

            train_loss+=loss.item() # record running loss

        # validation loop 
        model.eval()
        with torch.no_grad():
            for data, target in valid_loader:
                # uncoment for approach 2 resnet18 arhitecture
                #data = torch.stack([data,data,data], dim=1)
                
                target.squeeze_()
                loss = criterion(model(data), target)
                val_loss += loss.item()

        # average loss
        train_loss = train_loss/ len(train_loader)
        val_loss   = val_loss/ len(valid_loader)
        print('Epoch:{} \tTraining_Loss:{:.6}\tValidation_Loss:{:.6}'.format(epoch+1,train_loss,val_loss))
        
        # save  average loss across epochs
        train_plot.append(train_loss)
        val_plot.append(val_loss)
        
        # save model if validation loss has decreased
        if val_loss <= min_val_loss:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            min_val_loss,
            val_loss))
            print()
            torch.save(model.state_dict(), 'model.pt') 
            min_val_loss = val_loss
    
    return train_plot, val_plot

In [13]:
import torch 

def test(model, test_loader, criterion, classes:list):
    
    test_loss = 0.0
    class_correct = list(0. for i in range(len(classes))) 
    class_total = list(0. for i in range(len(classes)))
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            #data = torch.stack([data,data,data], dim=1)
            
            output = model(data)
            target.squeeze_()
            test_loss += criterion(output, target).item()
            probs = torch.exp(output)# get probabilities from model output output
            _,idx = torch.topk(probs, k=1, dim=1) # index of element with highest probability
            correct = idx == target.view(*idx.shape)
            
            accuracy += torch.mean(correct.type(torch.FloatTensor))
            
            # calculate test accuracy for each object class
            for i in range(target.size(0)):
                label = target[i]
                class_correct[label] += correct[i].item()
                class_total[label] += 1

        # calculate and print avg test loss
        test_loss = test_loss/len(test_loader)
        print('Test Loss: {:.4f}\t'.format(test_loss), end="")    
        print('\nTest Accuracy (Overall): {:.4f}'.format(accuracy.item()/len(test_loader)))
    
    return test_loss, accuracy, class_correct, class_total

In [14]:
def print_test_data(class_correct:list, class_total:list, classes:list):
    """
        class_correct:list -> returned by test function
        class_total:list -> returned by test function
        classes:list -> what are we classifying
    """
    for i in range(len(classes)):
        if class_total[i] > 0:
            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(classes[i]), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Test Accuracy of class %5s: N/A (no training examples)' % (i))